In [2]:
import os

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import (
    silhouette_score,
    davies_bouldin_score,
    adjusted_rand_score,
    normalized_mutual_info_score,
)

In [3]:
plt.style.use("default")
plt.rcParams["figure.figsize"]=(8,5)

In [4]:
DATA_PATH=os.path.join("..","data","raw","creditcard.csv")

df=pd.read_csv(DATA_PATH)

df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [5]:
df["Class"].value_counts(), df["Class"].value_counts(normalize=True)

(Class
 0    284315
 1       492
 Name: count, dtype: int64,
 Class
 0    0.998273
 1    0.001727
 Name: proportion, dtype: float64)

In [6]:
fraud_df=df[df["Class"]==1]
non_fraud_df=df[df["Class"]==0]

len(fraud_df), len(non_fraud_df)

(492, 284315)

In [8]:
non_fraud_sampled = non_fraud_df.sample(n=20000, random_state=42)

len(non_fraud_sampled)

20000

In [9]:
sampled_df = pd.concat([non_fraud_sampled, fraud_df], axis=0)

sampled_df = sampled_df.sample(frac=1.0, random_state=42).reset_index(drop=True)

sampled_df.shape, sampled_df["Class"].value_counts()

((20492, 31),
 Class
 0    20000
 1      492
 Name: count, dtype: int64)

In [11]:
PROCESSED_PATH = os.path.join("..","data","processed","creditcard_sampled.csv")

os.makedirs(os.path.dirname(PROCESSED_PATH), exist_ok=True)

sampled_df.to_csv(PROCESSED_PATH, index=False)

PROCESSED_PATH

'..\\data\\processed\\creditcard_sampled.csv'

In [12]:
X = sampled_df.drop(columns=["Class"])
y = sampled_df["Class"]
X.shape , y.shape

((20492, 30), (20492,))

(           Time        V1        V2        V3        V4        V5        V6  \
 0      133080.0  1.600491 -0.555101 -1.967577  0.719394 -0.068428 -1.365005   
 1       62867.0  1.112065  0.255520  0.427033  1.083135 -0.110176 -0.375269   
 2       50512.0 -1.601797 -0.251758  1.846010 -1.074837 -1.576391  0.785578   
 3       61289.0  0.641977 -0.852359 -0.117081  0.860912 -0.702012 -0.717593   
 4      158179.0 -0.315626  1.184693 -1.498383  0.751626  1.655667 -0.619278   
 ...         ...       ...       ...       ...       ...       ...       ...   
 20487   90731.0 -0.263602  0.894698 -0.876244 -0.214422  1.541905 -0.648319   
 20488   36157.0 -0.314511  1.350071  1.746420  3.184533  0.733771 -0.352377   
 20489   71116.0 -0.778253  0.130171  2.330368  1.743399 -0.602476  0.995023   
 20490   61944.0 -2.242599  0.091694  1.335811  0.234568  1.097002  0.989207   
 20491  167162.0  1.969201 -0.119482 -0.924658  1.481338  0.205727  0.145815   
 
              V7        V8        V9  